# 必要なライブラリのインポート

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
 
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn import preprocessing


# データセットの読み込み

In [2]:
wine = load_wine()
pd.DataFrame(wine.data, columns=wine.feature_names)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [3]:
wine.data

array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [4]:
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

In [5]:
train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size = 0.25)

# PyTorchへのテンソル（多次元行列）への変換

In [6]:
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

In [7]:
train=TensorDataset(train_X, train_Y)
train[0]

(tensor([1.1820e+01, 1.4700e+00, 1.9900e+00, 2.0800e+01, 8.6000e+01, 1.9800e+00,
         1.6000e+00, 3.0000e-01, 1.5300e+00, 1.9500e+00, 9.5000e-01, 3.3300e+00,
         4.9500e+02]),
 tensor(1))

In [8]:
train_loader = DataLoader(train, batch_size=15, shuffle=True)

# モデルの定義

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 128)
        self.fc2 = nn.Linear(128, 2)
         
    def forward(self, x):
        x = F.relu(self.fc1(x)) # ReLU: max(x, 0)
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [10]:
device = torch.device('cuda:0')

In [11]:
model = Net()
model.to(device)

Net(
  (fc1): Linear(in_features=13, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

# トレーニング

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
for epoch in range(500): #学習回数500回
    total_loss = 0
     
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        optimizer.zero_grad()
        output = model(train_x.to(device))
        loss = criterion(output, train_y.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() #loss.data[0]で記述するとPyTorch0.5以上ではエラーが返る
     
    if (epoch+1)%60 == 0:
        print(epoch+1, total_loss)

60 4.852030992507935
120 4.852030992507935
180 4.852030992507935
240 4.852030992507935
300 4.852030992507935
360 4.852030992507935
420 4.852030992507935
480 4.852030992507935


# 精度計算

In [13]:
test_x, test_y = Variable(test_X), Variable(test_Y)
result = model(test_x.to(device))
result

tensor([[-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)

In [14]:
result = torch.max(result, 1)[1]
result

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [15]:
result = result.to('cpu')
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
accuracy

0.5151515151515151

# ニューラルネットワークの多層化（ディープラーニング）による精度検証

In [16]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 2)
         
    def forward(self, x):
        x = F.relu(self.fc1(x)) # ReLU: max(x, 0)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=0)

In [17]:
model = Net()
model.to(device)

Net(
  (fc1): Linear(in_features=13, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=2, bias=True)
)

## トレーニング

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
for epoch in range(500): #学習回数500回
    total_loss = 0
     
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        optimizer.zero_grad()
        output = model(train_x.to(device))
        loss = criterion(output, train_y.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() #loss.data[0]で記述するとPyTorch0.5以上ではエラーが返る
     
    if (epoch+1)%60 == 0:
        print(epoch+1, total_loss)

60 1.3971904143691063
120 1.170633725821972
180 1.5128415897488594
240 0.676971796900034
300 1.291664656251669
360 1.66633265838027
420 1.0803389134816825
480 1.340087465941906


## 精度計算

In [19]:
test_x, test_y = Variable(test_X), Variable(test_Y)
result = torch.max(model(test_x.to(device)).data, 1)[1]
result = result.to('cpu')
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
accuracy

0.8787878787878788

In [20]:
result

tensor([0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 1, 0, 0, 1, 1, 0, 1, 1])

# 作り直し

## データセットの読み込み

In [21]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
train.shape, test.shape

((27100, 17), (18050, 16))

In [22]:
test['y'] = -999

In [23]:
all_df = pd.concat([train, test], axis=0)
all_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
id,,,,,,,,,,,,,,,,,
0,31,services,married,secondary,no,12294,yes,no,cellular,21,nov,101,3,498,0,other,0
1,29,entrepreneur,single,tertiary,no,43027,no,no,cellular,22,aug,158,2,702,0,unknown,1
2,35,management,married,tertiary,no,12252,yes,no,cellular,11,nov,351,1,826,0,failure,0
3,31,technician,married,secondary,no,99121,yes,yes,unknown,16,may,658,2,120,0,failure,0
4,48,unemployed,married,primary,no,42005,yes,no,telephone,3,apr,177,1,273,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,49,self-employed,married,tertiary,no,98357,yes,no,cellular,6,jul,101,2,417,0,failure,-999
18046,34,blue-collar,married,secondary,no,29621,yes,no,cellular,12,may,345,1,815,0,unknown,-999
18047,34,admin.,single,secondary,no,94260,yes,no,unknown,16,may,121,2,370,0,unknown,-999


In [24]:
cols_o = all_df.select_dtypes(include='object').columns.tolist()
cols_o

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome']

In [25]:
for column in cols_o:
    le = preprocessing.LabelEncoder()
    le.fit(all_df[column])
    all_df[column] = le.transform(all_df[column])

all_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
id,,,,,,,,,,,,,,,,,
0,31,7,1,1,0,12294,1,0,0,21,8,101,3,498,0,1,0
1,29,2,2,2,0,43027,0,0,0,22,1,158,2,702,0,3,1
2,35,4,1,2,0,12252,1,0,0,11,8,351,1,826,0,0,0
3,31,9,1,1,0,99121,1,1,2,16,7,658,2,120,0,0,0
4,48,10,1,0,0,42005,1,0,1,3,0,177,1,273,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,49,6,1,2,0,98357,1,0,0,6,4,101,2,417,0,0,-999
18046,34,1,1,1,0,29621,1,0,0,12,7,345,1,815,0,3,-999
18047,34,0,2,1,0,94260,1,0,2,16,7,121,2,370,0,3,-999


In [26]:
X = all_df[all_df['y'] != -999].drop('y', axis=1).values
y = all_df[all_df['y'] != -999]['y'].values
X.shape, y.shape

((27100, 16), (27100,))

In [27]:
X

array([[ 31,   7,   1, ..., 498,   0,   1],
       [ 29,   2,   2, ..., 702,   0,   3],
       [ 35,   4,   1, ..., 826,   0,   0],
       ...,
       [ 35,   7,   1, ..., 121,   0,   3],
       [ 30,   0,   2, ..., 100,   0,   3],
       [ 34,   4,   1, ..., 321,   0,   3]], dtype=int64)

In [54]:
train_X, test_X, train_Y, test_Y = train_test_split(X, y, test_size = 0.25)
train_X.shape, test_X.shape, train_Y.shape, test_Y.shape

((20325, 16), (6775, 16), (20325,), (6775,))

## PyTorchへのテンソル（多次元行列）への変換

In [55]:
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

In [56]:
train=TensorDataset(train_X, train_Y)
train[0]

(tensor([3.5000e+01, 4.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 7.2690e+04,
         1.0000e+00, 0.0000e+00, 0.0000e+00, 2.7000e+01, 7.0000e+00, 3.4500e+02,
         2.0000e+00, 4.9800e+02, 0.0000e+00, 3.0000e+00]),
 tensor(0))

In [57]:
train_loader = DataLoader(train, batch_size=15, shuffle=True)

In [62]:
train_X.shape

torch.Size([20325, 16])

## モデルの定義

In [63]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(16, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 2)
         
    def forward(self, x):
        x = F.relu(self.fc1(x)) # ReLU: max(x, 0)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=0)

In [64]:
model = Net()
# model.to(device)

## トレーニング

In [65]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
for epoch in range(500): #学習回数500回
    total_loss = 0
     
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        optimizer.zero_grad()
        output = model(train_x)
#         output = model(train_x.to(device))
        print(output)
        loss = criterion(output, train_y)
#         loss = criterion(output, train_y.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() #loss.data[0]で記述するとPyTorch0.5以上ではエラーが返る
     
    if (epoch+1)%60 == 0:
        print(epoch+1, total_loss)

tensor([[-6.3029e+01, -1.1301e+02],
        [-8.0592e+01, -1.4602e+02],
        [-6.3943e+01, -1.1369e+02],
        [-8.6352e+01, -1.5622e+02],
        [-2.4973e-05, -5.2834e-09],
        [-2.5768e+01, -4.6132e+01],
        [-1.0670e+01, -1.9074e+01],
        [-1.8261e+01, -3.1909e+01],
        [-8.2669e+01, -1.4943e+02],
        [-4.7318e+01, -8.6465e+01],
        [-6.1114e+01, -1.1006e+02],
        [-6.9346e+01, -1.2573e+02],
        [-1.3272e+01, -2.3275e+01],
        [-7.2026e+01, -1.3038e+02],
        [-4.6395e+01, -8.3346e+01]], grad_fn=<LogSoftmaxBackward>)
tensor([[-9.2852e+05, -3.9761e+06],
        [-1.1114e+03, -4.8986e+06],
        [-4.5058e+06, -4.1795e+05],
        [-4.9260e+06,  0.0000e+00],
        [-4.8731e+06, -5.2604e+04],
        [-4.2148e+06, -7.0734e+05],
        [-3.2066e+06, -1.7102e+06],
        [-3.2913e+06, -1.6260e+06],
        [-1.2859e+05, -4.7718e+06],
        [-3.4554e+06, -1.4627e+06],
        [-1.4337e+05, -4.7571e+06],
        [-1.8952e+06, -3.0146e+06

        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
       

        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
       

        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
       

        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
       

...

        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
       

        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
       

        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
       

tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<LogSoftmaxBackward>)
tensor(

KeyboardInterrupt: 

## 精度計算

In [ ]:
test_x, test_y = Variable(test_X), Variable(test_Y)
result = torch.max(model(test_x.to(device)).data, 1)[1]
result = result.to('cpu')
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
accuracy

In [ ]:
result